# Download [zip_code_market_tracker.tsv000](https://www.kaggle.com/datasets/thuynyle/redfin-housing-market-data?select=zip_code_market_tracker.tsv000)

In [9]:
import pandas as pd
import polars as pl

In [10]:
!awk -F'\t' 'BEGIN {OFS="\t"} {NF=58; print}' zip_code_market_tracker.tsv000 > cleaned_data.tsv

In [11]:
charlotte_df = pl.scan_csv("cleaned_data.tsv", separator="\t").filter(
    (pl.col("parent_metro_region")  == 'Charlotte, NC') & 
    (pl.col("property_type") == 'All Residential'))

In [12]:
# Select final features
features_to_keep = [
    # Identifiers
    'period_begin', 'period_end', 'region', 'state_code',
    
    # Target
    'median_sale_price',
    
    # Core features (Tier 1)
    'median_list_price', 'median_ppsf',
    'homes_sold', 'inventory', 'new_listings', 'months_of_supply',
    'median_dom', 'pending_sales',
    'avg_sale_to_list', 'sold_above_list',
    'price_drops', 'off_market_in_two_weeks',
    
    # Pre-calculated trends (Tier 1)
    'median_sale_price_mom', 'median_sale_price_yoy',
    'months_of_supply_mom', 'months_of_supply_yoy',
    
    # Additional momentum (Tier 2)
    'homes_sold_mom', 'homes_sold_yoy',
    'inventory_mom', 'inventory_yoy',
    'median_dom_mom', 'median_dom_yoy',
    'price_drops_mom', 'price_drops_yoy',
    'pending_sales_mom', 'pending_sales_yoy',
    'new_listings_mom', 'new_listings_yoy',
]

# Filter and select
charlotte_clean = charlotte_df.select(features_to_keep)
charlotte_clean = charlotte_clean.with_columns(
    pl.col("region").str.replace("ZIP Code: ", "")
)

In [13]:
# Save
charlotte_clean.collect().write_csv("charlotte_cleaned_data.csv")

In [14]:
!rm cleaned_data.tsv